In [23]:
import pandas as pd
import numpy as np

### Preprocessing

In [24]:
data = pd.read_csv('twitter_data/tweets_extended_fields.csv')
data

/var/folders/pv/j2wxhdx50y5fbwqrc2yxqwf00000gn/T/ipykernel_64834/2236879572.py:1: DtypeWarning: Columns (14,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('twitter_data/tweets_extended_fields.csv')


,possibly_sensitive,conversation_id,in_reply_to_user_id,author_id,Tweets,public_metrics,source,lang,referenced_tweets,created_at,...,poll_ids,mentions,annotations,urls,hashtags,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags
0,False,1005377380997828608,1.843782e+09,54502344,"@castleburyexec Hi Andy, we want to help you h...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100537738099782...",2018-06-09T16:27:21.000Z,...,NaN,{},"[{'start': 19, 'end': 22, 'probability': 0.692...",{},{},NaN,['Person'],['Andy'],[],[]
1,False,1005253183902543872,7.746454e+17,54502344,@neo_ysl GLG 2020. It is yet to be confirm whe...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100527567064836...",2018-06-09T16:14:04.000Z,...,NaN,"[{'start': 0, 'end': 8, 'username': 'neo_ysl',...","[{'start': 9, 'end': 16, 'probability': 0.7305...",{},{},NaN,['Other'],['GLG 2020'],['neo_ysl'],[]
2,False,988516398077886464,2.234039e+07,54502344,@aWmeown Thanks for your contact. We're sorry ...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Unymira Connect,en,"[{'type': 'replied_to', 'id': '100526911657861...",2018-06-09T16:04:27.000Z,...,NaN,{},{},{},{},NaN,[],[],[],[]
3,False,1005201522328649728,2.755694e+08,50883209,@otaubman We want to get to the bottom of this...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",Customer online social care.,en,"[{'type': 'replied_to', 'id': '100520152232864...",2018-06-09T16:01:12.000Z,...,NaN,"[{'start': 0, 'end': 9, 'username': 'otaubman'...",{},"[{'start': 67, 'end': 90, 'url': 'https://t.co...",{},NaN,[],[],['otaubman'],[]
4,False,1005479690621026306,NaN,54502344,The Berlin State of Mind.\n\nEnjoy the true de...,"{'retweet_count': 95, 'reply_count': 7, 'like_...",erased6981162,en,{},2018-06-09T16:00:13.000Z,...,NaN,{},"[{'start': 4, 'end': 15, 'probability': 0.3684...","[{'start': 72, 'end': 95, 'url': 'https://t.co...","[{'start': 119, 'end': 131, 'tag': 'MBsocialca...",NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642892,False,998980817811582976,4.185260e+08,1545994664,@CianHogan2 Thank you for your interest in BMW...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Sprinklr,en,"[{'type': 'replied_to', 'id': '998980817811582...",2018-05-22T21:42:26.000Z,...,{},"[{'start': 0, 'end': 11, 'username': 'CianHoga...","[{'start': 43, 'end': 55, 'probability': 0.728...","[{'start': 211, 'end': 234, 'url': 'https://t....",{},NaN,"['Other', 'Other']","['BMW Connected', 'BMW Connected']",['CianHogan2'],[]
642893,False,999009662685986816,NaN,30981244,Are your tires properly inflated? Now is a goo...,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",Emplifi,en,{},2018-05-22T19:30:38.000Z,...,{},{},"[{'start': 64, 'end': 85, 'probability': 0.478...","[{'start': 87, 'end': 110, 'url': 'https://t.c...","[{'start': 63, 'end': 86, 'tag': 'NationalTire...",NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek']
642894,False,998998679565451265,NaN,121825214,In order to sustain our planet's finite resour...,"{'retweet_count': 4, 'reply_count': 0, 'like_c...",Twitter Ads,en,{},2018-05-22T18:47:00.000Z,...,{},{},"[{'start': 212, 'end': 222, 'probability': 0.5...","[{'start': 235, 'end': 258, 'url': 'https://t....","[{'start': 163, 'end': 194, 'tag': 'Innovation...",NaN,['Other'],['UTECHEurope'],[],"['InnovationForSustainableGrowth', 'UTECHEurope']"
642895,False,998980027197800449,NaN,30981244,This year's #Indy500 pole sitter @edcarpenter2...,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",Emplifi,en,{},2018-05-22T17:32:53.000Z,...,{},"[{'start': 33, 'end': 47, 'username': 'edcarpe...","[{'start': 13, 'end': 19, 'probability': 0.427...","[{'start': 203, 'end': 226, 'url': 'https://t....","[{'start

In [25]:
#format retweets data
retweet_ids = pd.read_csv('twitter_data/source_tweet_additional_text.csv')
retweet_ids['tweet_id'] = retweet_ids['tweet_id'].apply(lambda x: format(float(x), '.0f'))
retweet_ids['tweet_id'] = retweet_ids['tweet_id'].astype('int64')
retweet_ids = retweet_ids.rename(columns={"tweet_id": "id"})
retweet_ids = retweet_ids.drop(columns=['Unnamed: 0'])

In [26]:
data = data.merge(retweet_ids, how="left", on="id")

In [27]:
data['full_tweet'] = np.where(data['sourcetweet_text'].astype(str).str.len()< 5, data['Tweets'], data['sourcetweet_text'])
data = data.rename(columns={"Tweets": "tweets_original","full_tweet": "Tweets"})

In [28]:
data['year'] = pd.to_datetime(data['created_at']).dt.year
data['more_information_provided'] = (~data['urls'].isna())&~(data['urls']=="{}")

In [29]:
data = data.drop(columns=['public_metrics','author_id','attachments','entities','referenced_tweets','edit_history_tweet_ids','conversation_id','attachments','geo','hashtags','annotations','mentions'])

In [30]:
#remove other languages (23881 posts were non English and non German)
data = data[(data['lang'] == 'en') | (data['lang']=='de')]
data

,possibly_sensitive,in_reply_to_user_id,tweets_original,source,lang,created_at,id,author,Branche,Branche-6,...,urls,cashtags,entity_types,entity_names,mentioned_usernames,mentioned_hashtags,sourcetweet_text,Tweets,year,more_information_provided
0,False,1.843782e+09,"@castleburyexec Hi Andy, we want to help you h...",Unymira Connect,en,2018-06-09T16:27:21.000Z,1005486517891067906,MercedesBenz,automobil,Automobil,...,{},NaN,['Person'],['Andy'],[],[],NaN,"@castleburyexec Hi Andy, we want to help you h...",2018,False
1,False,7.746454e+17,@neo_ysl GLG 2020. It is yet to be confirm whe...,Unymira Connect,en,2018-06-09T16:14:04.000Z,1005483173244030976,MercedesBenz,automobil,Automobil,...,{},NaN,['Other'],['GLG 2020'],['neo_ysl'],[],NaN,@neo_ysl GLG 2020. It is yet to be confirm whe...,2018,False
2,False,2.234039e+07,@aWmeown Thanks for your contact. We're sorry ...,Unymira Connect,en,2018-06-09T16:04:27.000Z,1005480753512185856,MercedesBenz,automobil,Automobil,...,{},NaN,[],[],[],[],NaN,@aWmeown Thanks for your contact. We're sorry ...,2018,False
3,False,2.755694e+08,@otaubman We want to get to the bottom of this...,Customer online social care.,en,2018-06-09T16:01:12.000Z,1005479935526555657,PUMA,mode,Textil,...,"[{'start': 67, 'end': 90, 'url': 'https://t.co...",NaN,[],[],['otaubman'],[],NaN,@otaubman We want to get to the bottom of this...,2018,True
4,False,NaN,The Berlin State of Mind.\n\nEnjoy the true de...,erased6981162,en,2018-06-09T16:00:13.000Z,1005479690621026306,MercedesBenz,automobil,Automobil,...,"[{'start': 72, 'end': 95, 'url': 'https://t.co...",NaN,"['Organization', 'Other', 'Person', 'Organizat...","['Berlin State', 'Mind', 'Constantin Schiller'...",[],"['MBsocialcar', 'MercedesBenz']",NaN,The Berlin State of Mind.\n\nEnjoy the true de...,2018,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642892,False,4.185260e+08,@CianHogan2 Thank you for your interest in BMW...,Sprinklr,en,2018-05-22T21:42:26.000Z,999042828746575875,BMW,automobil,Automobil,...,"[{'start': 211, 'end': 234, 'url': 'https://t....",NaN,"['Other', 'Other']","['BMW Connected', 'BMW Connected']",['CianHogan2'],[],NaN,@CianHogan2 Thank you for your interest in BMW...,2018,True
642893,False,NaN,Are your tires properly inflated? Now is a goo...,Emplifi,en,2018-05-22T19:30:38.000Z,999009662685986816,continentaltire,automobil,Automobil,...,"[{'start': 87, 'end': 110, 'url': 'https://t.c...",NaN,['Other'],['NationalTireSafetyWeek'],[],['NationalTireSafetyWeek'],NaN,Are your tires properly inflated? Now is a goo...,2018,True
642894,False,NaN,In order to sustain our planet's finite resour...,Twitter Ads,en,2018-05-22T18:47:00.000Z,998998679565451265,covestro,chemie,Technik,...,"[{'start': 235, 'end': 258, 'url': 'https://t....",NaN,['Other'],['UTECHEurope'],[],"['InnovationForSustainableGrowth', 'UTECHEurope']",NaN,In order to sustain our planet's finite resour...,2018,True
642895,False,NaN,This year's #Indy500 pole sitter @edcarpenter2...,Emplifi,en,2018-05-22T17:32:53.000Z,998980027197800449,continentaltire,automobil,Automobil,...,"[{'start': 203, 'end': 226, 'url': 'https://t....",NaN,['Other'],['Indy500'],"['edcarpenter20', 'DWRshow', 'IMS']",['Indy500'],NaN,This year's #Indy500 pole sitter @edcarpenter2...,2018,True


In [31]:
#removed all replies
data = data[data["in_reply_to_user_id"].isna()]
data = data.drop(columns=['in_reply_to_user_id','reply_ids'])

In [32]:
data['post_text'] = data['Tweets']
data = data.drop(columns=['Tweets'])

In [33]:
data.to_csv('twitter_data/posts_restructured.csv', header=True, index=False, columns=list(data.axes[1]), lineterminator='\r\n')